In [3]:
!pip install mysql-connector-python




SyntaxError: invalid syntax (2162649465.py, line 4)

In [14]:
#connecting to database
import mysql.connector as msc
conn = msc.connect(
    host="localhost",
    user="root",
    password="wasd",
    database="ir_policy"
)

In [17]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, pipeline

def summarise_pegasus(input_text):
    model_name = 'google/pegasus-large'

    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name)

    summary_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)
    summary = summary_pipeline(input_text, max_length=300, min_length=50, truncation=True)

    return summary[0]['summary_text']


In [6]:
import re
import spacy
from nltk.tokenize import sent_tokenize

# Load spaCy model for NER
nlp = spacy.load("en_core_web_sm")

# Ensure you have the necessary NLTK data
import nltk
nltk.download('punkt')

def preprocess_text(text):
    """
    Preprocess the text for the PEGASUS summarization model, focusing on handling named entities
    and intelligent chunking for terms and conditions simplification. Named entities are preserved
    in their original form, and the output is a single string of text suitable for summarization.

    Args:
    text (str): The input text to preprocess.

    Returns:
    str: The preprocessed text ready for the PEGASUS model.
    """

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Decode HTML entities (e.g., '&amp;' to '&')
    text = re.sub(r'&[a-z]+;', '', text)

    # Temporarily replace named entities with placeholders to avoid splitting them
    doc = nlp(text)
    entities = {}
    for ent in doc.ents:
        placeholder = f"__{ent.label_}__"
        entities[placeholder] = ent.text
        text = text.replace(ent.text, placeholder)

    # Remove special characters, keeping alphabets, common punctuation, and placeholders for entities
    text = re.sub(r'[^a-zA-Z.,;!?\'"__\s]', ' ', text)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # Split the text into sentences and rejoin to ensure proper sentence boundaries
    sentences = sent_tokenize(text)
    text = ' '.join(sentences)

    # Restore named entities in the text by replacing placeholders with original entities
    for placeholder, original in entities.items():
        text = text.replace(placeholder, original)

    return text


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leifo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS generated_data (
        appid INT PRIMARY KEY,
        summary TEXT
    );
""")
conn.commit()


In [19]:
insert_sql = """
INSERT INTO ir_policy.generated_data (appid, summary)
VALUES (%s, %s)
ON DUPLICATE KEY UPDATE summary = VALUES(summary);
"""


In [9]:
'''cursor.execute("SELECT * FROM ir_policy.apps_data")

try:
    while True:
        row = cursor.fetchone()
        if row is None:
            break
        print(row)
        
        # The rest of your logic here
        # ...

except mysql.connector.Error as err:
    print("Error:", err)
finally:
    cursor.close()
    conn.close()
'''

(1, 'Texting', 'Whatsapp', 'Skip to content\nWhatsApp Main Page\n\nFeatures\nPrivacy\nHelp Center\nBlog\nWhatsApp Web\nDownload\nLast modified: January 4, 2021 (archived versions)\n\nWhatsApp Privacy Policy\nIf you live in the European Region, WhatsApp Ireland Limited provides the services to you under this Terms of Service and Privacy Policy.\n\nWhatsApp Legal Info\nIf you live outside the European Region, WhatsApp LLC ("WhatsApp," "our," "we," or "us") provides our Services to you under this Terms of Service and Privacy Policy.\n\nOur Privacy Policy ("Privacy Policy") helps explain our data practices, including the information we process to provide our Services.\n\nFor example, our Privacy Policy talks about what information we collect and how this affects you. It also explains the steps we take to protect your privacy, like building our Services so delivered messages aren’t stored by us and giving you control over who you communicate with on our Services.\n\nWe are one of the Meta C

In [20]:
cursor.execute("SELECT * FROM ir_policy.apps_data")
rows = cursor.fetchall()  # Fetch all rows at once

for row in rows:
    appid = row[0]
    input_text = row[3]

    # Preprocess the input text and summarize
    preprocessed_txt = preprocess_text(input_text)
    tnc_summary = summarise_pegasus(preprocessed_txt)

    # Step 2 & 3: Insert or update the summary in generated_data
    # Use a different cursor for the write operations
    write_cursor = conn.cursor()
    write_cursor.execute(insert_sql, (appid, tnc_summary))
    conn.commit()  # Commit each insert
    write_cursor.close()

cursor.close()  # Close the original cursor
conn.close()


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a